In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [3]:
# Load the trained model and encoders
model = load_model('model.h5')
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)
with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [4]:
def preprocess_input(data):
    data_df = pd.DataFrame([data])
    
    # Encode 'Gender'
    data_df['Gender'] = label_encoder_gender.transform(data_df['Gender'])
    
    # One-hot encode 'Geography'
    geo_encoded = onehot_encoder_geo.transform(data_df[['Geography']]).toarray()
    geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
    data_df = pd.concat([data_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
    
    # Scale the features
    data_scaled = scaler.transform(data_df)
    
    return data_scaled

In [5]:

# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}


In [6]:
# Preprocess the input data and predict churn
input_scaled = preprocess_input(input_data)
prediction = model.predict(input_scaled)
prediction_proba = prediction[0][0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step


In [7]:
# Output the prediction result
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
